In [ ]:
from keras import backend as K
from keras.layers import Activation, Dense, Input, Subtract
from keras.models import Model
import numpy as np
import math

In [65]:
class RankerNN(object):

    def __init__(self, input_size, hidden_layer_sizes=(100,), activation=('relu',), solver='adam'):
        if len(hidden_layer_sizes) != len(activation):
            raise ValueError('hidden_layer_sizes and activation should have the same size.')
        self.model = self._build_model(input_size, hidden_layer_sizes, activation)
        self.model.compile(optimizer=solver, loss="binary_crossentropy")

    @staticmethod
    def _build_model(input_shape, hidden_layer_sizes, activation):
        # Neural network structure
        hidden_layers = []
        for i in range(len(hidden_layer_sizes)):
            hidden_layers.append(Dense(hidden_layer_sizes[i], activation=activation[i], name=str(activation[i]) + '_layer' + str(i)))
        h0 = Dense(1, activation='linear', name='Identity_layer')
        input1 = Input(shape=(input_shape,), name='Input_layer1')
        input2 = Input(shape=(input_shape,), name='Input_layer2')
        x1 = input1
        x2 = input2
        for i in range(len(hidden_layer_sizes)):
            x1 = hidden_layers[i](x1)
            x2 = hidden_layers[i](x2)
        x1 = h0(x1)
        x2 = h0(x2)
        # Subtract layer
        subtracted = Subtract(name='Subtract_layer')([x1, x2])
        # sigmoid
        out = Activation('sigmoid', name='Activation_layer')(subtracted)
        # build model
        model = Model(inputs=[input1, input2], outputs=out)
        return model

    @staticmethod
    def _CalcDCG(labels):
        sumdcg = 0.0
        for i in range(len(labels)):
            rel = labels[i]
            if rel != 0:
                sumdcg += ((2 ** rel) - 1) / math.log2(i + 2)
        return sumdcg

    def _fetch_qid_data(self, y, qid, eval_at=None):
        qid_unique, qid2indices, qid_inverse_indices = np.unique(qid, return_index=True, return_inverse=True)
        # get item releveance for each query id
        qid2rel = [[] for _ in range(len(qid_unique))]
        for i, qid_unique_index in enumerate(qid_inverse_indices):
            qid2rel[qid_unique_index].append(y[i])
        # get dcg, idcg for each query id @eval_at
        if eval_at:
            qid2dcg = [self._CalcDCG(qid2rel[i][:eval_at]) for i in range(len(qid_unique))]
            qid2idcg = [self._CalcDCG(sorted(qid2rel[i], reverse=True)[:eval_at]) for i in range(len(qid_unique))]
        else:
            qid2dcg = [self._CalcDCG(qid2rel[i]) for i in range(len(qid_unique))]
            qid2idcg = [self._CalcDCG(sorted(qid2rel[i], reverse=True)) for i in range(len(qid_unique))]
        return qid2indices, qid2rel, qid2idcg, qid2dcg


    def _transform_pairwise(self, X, y, qid):
        return None, None, None, None


    def fit(self, X, y, qid, batch_size=None, epochs=1, verbose=1, validation_split=0.0):
        X1_trans, X2_trans, y_trans, weight = self._transform_pairwise(X, y, qid)
        self.model.fit([X1_trans, X2_trans], y_trans, sample_weight=weight, batch_size=batch_size, epochs=epochs,
                       verbose=verbose, validation_split=validation_split)
        self.evaluate(X, y, qid)

    def predict(self, X):
        ranker_output = K.function([self.model.layers[0].input], [self.model.layers[-3].get_output_at(0)])
        return ranker_output([X])[0].ravel()

    def evaluate(self, X, y, qid, eval_at=None):
        y_pred = self.predict(X)
        tmp = np.array(np.hstack([y.reshape(-1, 1), y_pred.reshape(-1, 1), qid.reshape(-1, 1)]))
        tmp = tmp[np.lexsort((-tmp[:, 1], tmp[:, 2]))]
        y_sorted = tmp[:, 0]
        qid_sorted = tmp[:, 2]
        ndcg = self._EvalNDCG(y_sorted, qid_sorted, eval_at)
        if eval_at:
            print('ndcg@' + str(eval_at) + ': ' + str(ndcg))
        else:
            print('ndcg: ' + str(ndcg))

    def _EvalNDCG(self, y, qid, eval_at=None):
        _, _, qid2idcg, qid2dcg = self._fetch_qid_data(y, qid, eval_at)
        sumndcg = 0
        count = 0.0
        for qid_unique_idx in range(len(qid2idcg)):
            count += 1
            if qid2idcg[qid_unique_idx] == 0:
                continue
            idcg = qid2idcg[qid_unique_idx]
            dcg = qid2dcg[qid_unique_idx]
            sumndcg += dcg / idcg
        return sumndcg / count


class RankNetNN(RankerNN):

    def __init__(self, input_size, hidden_layer_sizes=(100,), activation=('relu',), solver='adam'):
        super(RankNetNN, self).__init__(input_size, hidden_layer_sizes, activation, solver)

    def _transform_pairwise(self, X, y, qid):
        qid2indices, qid2rel, qid2idcg, _ = self._fetch_qid_data(y, qid)
        X1 = []
        X2 = []
        weight = []
        Y = []
        for qid_unique_idx in range(len(qid2indices)):
            if qid2idcg[qid_unique_idx] == 0: # 해당 쿼리의 idcg가 0인 경우 : relevance score가 존재하는 point가 하나도 없는 경우
                continue
            rel_list = qid2rel[qid_unique_idx] # 해당 쿼리의 rel_list
            qid_start_idx = qid2indices[qid_unique_idx] 
            for pos_idx in range(len(rel_list)): # 이중 for loop 으로, pair(i, j)를 iteration (pos:i, neg:j)
                for neg_idx in range(len(rel_list)):
                    if rel_list[pos_idx] <= rel_list[neg_idx]: # i번째 rel이 j보다 큰 경우만 경우를 따짐 (쇼당은 안붙음)
                        continue
                    # balanced class : pos의 index가 neg의 index보다 큰 경우
                    if 1 != (-1) ** (qid_unique_idx + pos_idx + neg_idx):
                        X1.append(X[qid_start_idx + pos_idx])
                        X2.append(X[qid_start_idx + neg_idx])
                        weight.append(1)
                        Y.append(1)
                    else:
                        X1.append(X[qid_start_idx + neg_idx])
                        X2.append(X[qid_start_idx + pos_idx])
                        weight.append(1)
                        Y.append(0)
        return np.asarray(X1), np.asarray(X2), np.asarray(Y), np.asarray(weight)


class LambdaRankNN(RankerNN):

    def __init__(self, input_size, hidden_layer_sizes=(100,), activation=('relu',), solver='adam'):
        super(LambdaRankNN, self).__init__(input_size, hidden_layer_sizes, activation, solver)

    def _transform_pairwise(self, X, y, qid):
        qid2indices, qid2rel, qid2idcg, _ = self._fetch_qid_data(y, qid)
        X1 = []
        X2 = []
        weight = []
        Y = []
        for qid_unique_idx in range(len(qid2indices)):
            if qid2idcg[qid_unique_idx] == 0:
                continue
            IDCG = 1.0 / qid2idcg[qid_unique_idx]
            rel_list = qid2rel[qid_unique_idx]
            qid_start_idx = qid2indices[qid_unique_idx]
            for pos_idx in range(len(rel_list)):
                for neg_idx in range(len(rel_list)):
                    if rel_list[pos_idx] <= rel_list[neg_idx]:
                        continue
                        
                        
                    """"""
                    # calculate lambda
                    pos_loginv = 1.0 / math.log2(pos_idx + 2) # ???
                    neg_loginv = 1.0 / math.log2(neg_idx + 2) # ???
                    pos_label = rel_list[pos_idx] # 
                    neg_label = rel_list[neg_idx] # ???
                    original = ((2 ** pos_label) - 1) * pos_loginv + ((2 ** neg_label) - 1) * neg_loginv
                    changed = ((2 ** neg_label) - 1) * pos_loginv + ((2 ** pos_label) - 1) * neg_loginv
                    delta = (original - changed) * IDCG
                    if delta < 0:
                        delta = -delta
                    """"""
                        
                        
                        
                    # balanced class
                    if 1 != (-1) ** (qid_unique_idx + pos_idx + neg_idx):
                        X1.append(X[qid_start_idx + pos_idx])
                        X2.append(X[qid_start_idx + neg_idx])
                        weight.append(delta)
                        Y.append(1)
                    else:
                        X1.append(X[qid_start_idx + neg_idx])
                        X2.append(X[qid_start_idx + pos_idx])
                        weight.append(delta)
                        Y.append(0)
        return np.asarray(X1), np.asarray(X2), np.asarray(Y), np.asarray(weight)

In [66]:
import numpy as np

# query feature
X = np.array([[0.2, 0.3, 0.4],
              [0.1, 0.7, 0.4],
              [0.3, 0.4, 0.1],
              [0.8, 0.4, 0.3],
              [0.9, 0.35, 0.25]])
y = np.array([0, 1, 0, 0, 2]) # 쿼리의 relevance score (ideal rank : left to right)
qid = np.array([1, 1, 1, 2, 2]) # 쿼리 구분

# train model
ranker = RankNetNN(input_size=X.shape[1], hidden_layer_sizes=(16,8,), activation=('relu', 'relu',), solver='adam')
ranker.fit(X, y, qid, epochs=5)
y_pred = ranker.predict(X)
ranker.evaluate(X, y, qid, eval_at=2)

-------------
qid: 0
pos: 0 0
neg: 0 0
-------------
-------------
qid: 0
pos: 0 0
neg: 1 1
-------------
-------------
qid: 0
pos: 0 0
neg: 2 0
-------------
-------------
qid: 0
pos: 1 1
neg: 0 0
-------------
-------------
bal qid: 0
bal pos: 1 1
bal neg: 0 0
1
-1
-------------
-------------
qid: 0
pos: 1 1
neg: 1 1
-------------
-------------
qid: 0
pos: 1 1
neg: 2 0
-------------
-------------
bal qid: 0
bal pos: 1 1
bal neg: 2 0
3
-1
-------------
-------------
qid: 0
pos: 2 0
neg: 0 0
-------------
-------------
qid: 0
pos: 2 0
neg: 1 1
-------------
-------------
qid: 0
pos: 2 0
neg: 2 0
-------------
-------------
qid: 1
pos: 0 0
neg: 0 0
-------------
-------------
qid: 1
pos: 0 0
neg: 1 2
-------------
-------------
qid: 1
pos: 1 2
neg: 0 0
-------------
-------------
bal qid: 1
bal pos: 1 2
bal neg: 0 0
2
1
-------------
-------------
qid: 1
pos: 1 2
neg: 1 2
-------------
Epoch 1/5
3/3 [==============================] - 0s 103ms/step - loss: 0.7606
Epoch 2/5
3/3 [=========

In [50]:
qid = np.array([1, 1, 1, 2, 2])
qid_unique, qid2indices, qid_inverse_indices = np.unique(qid, return_index=True, return_inverse=True)
print(qid_unique)
print("__---___----")
print(qid2indices)
print("__---___----")
print(qid_inverse_indices)
print("__---___----")

[1 2]
__---___----
[0 3]
__---___----
[0 0 0 1 1]
__---___----


In [51]:
qid2rel = [[] for _ in range(len(qid_unique))]

In [52]:
qid2rel

[[], []]

In [54]:
print([qid2rel[i][:2] for i in range(len(qid_unique))])
print([sorted(qid2rel[i], reverse=True)[:2] for i in range(len(qid_unique))])

[[], []]
[[], []]


In [48]:
(-1) ** (0 + 1 + 2)

-1

In [71]:
print(1 << 3)
print(2 ** 3)

8
8


In [72]:
print(1 << 4)
print(2 ** 4)

16
16


In [73]:
print(1 << 2)
print(2 ** 2)

4
4


In [70]:
1 << 1

2